<a href="https://colab.research.google.com/github/kavyajeetbora/geemap/blob/master/notebooks/02_geemap_using_basemaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install geemap

uncomment this code if not already installed

In [ ]:
# %pip install -U geemap

## Import libraries

In [42]:
import ee
import geemap

## Earth Engine Authentication and Initialization

In [27]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=9vtsRDwhmp9y7LHC3ymWplPk21BqzZVgkeO30vPaVfs&tc=XlvU1EP9l7NjmRKa_Qsuhe-o0q8Dcg8dMJtzncIjQKU&cc=xcu8nwABq4mTbrSPvZYzU9Zx1Nh9kzZMa4qPjEB-tfo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkoz760WYjG9stxDL_3R_Afex105BoGQGxSbdQ3HuEJ5zGHxzVYUvw

Successfully saved authorization token.


## Create an interactive map

In [44]:
geemap.show_youtube('6J5ZCIUPXfI')

Output(layout=Layout(width='815px'))

In [43]:
m = geemap.Map(center=(40,-100), zoom=4)
m.add_basemap('HYBRID')
m.add_basemap('OpenTopoMap')

# m.basemap_demo()
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Add WMS and XYZ tile layers




In [36]:
Map = geemap.Map()
# https://viewer.nationalmap.gov/services/
url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Satellite', attribution='Google')

naip_url = 'https://services.nationalmap.gov/arcgis/services/USGSNAIPImagery/ImageServer/WMSServer?'
Map.add_wms_layer(
    url=naip_url, layers='0', name='NAIP Imagery', format='image/png', shown=True
)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Add Earth Engine data layers

In [37]:
Map = geemap.Map()
# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
states = ee.FeatureCollection("TIGER/2018/States")

# Set visualization parameters.
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine layers to Map
Map.addLayer(dem, vis_params, 'SRTM DEM', True, 0.5)
Map.addLayer(landcover, {}, 'Land cover')
Map.addLayer(
    landsat7, {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200}, 'Landsat 7'
)
Map.addLayer(states, {}, "US States")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

##  Search Earth Engine data catalog

In [40]:
Map = geemap.Map()
location = Map.search_loc_geom
print(location.getInfo())

AttributeError: ignored

## Feature collection

In [41]:
geemap.ee_search()

## Clipping raster withing a drawn geometry

ou can draw shapes on the map using the draw control. The drawn features will be automatically converted to Earth Engine objects, which can be accessed in one of the following ways:

In [18]:
m = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': 'terrain',
}
m.add_layer(dem, vis_params, 'SRTM DEM')
m.add('layer_manager')
m

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

To return the last drawn feature as an ee.Geometry(), use: `m._draw_control.last_geometry`

In [22]:
geometry = m._draw_control.last_geometry
geometry

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -92.329102,
              40.5472
            ],
            [
              -94.174805,
              37.614231
            ],
            [
              -93.076172,
              35.496456
            ],
            [
              -89.25293,
              34.234512
            ],
            [
              -92.329102,
              40.5472
            ]
          ]
        ]
      },
      "geodesic": {
        "constantValue": false
      }
    }
  }
})

To return all the drawn feature as an ee.Feature(), use: `m._draw_control.last_feature`

In [23]:
feature = m._draw_control.last_feature
feature

To return all the drawn features as an ee.FeatureCollection(), use: `m._draw_control.collection`

In [24]:
collection = m._draw_control.collection
collection

Clip the DEM data using the drawn polygon.

In [25]:
if geometry is not None:
    image = dem.clipToCollection(collection)
    m.layers[1].visible = False
    m.add_layer(image, vis_params, "Clipped DEM")
m

Map(bottom=3395.0, center=[40.212440718286466, -85.47363281250001], controls=(ZoomControl(options=['position',…